In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobadlskey"))

In [0]:
hospital_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/hospital.csv", header=True, inferSchema=False)

In [0]:
hospital_data.show(5,False)

In [0]:
#hospital_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in hospital_data.columns]).show()

In [0]:
hospital_data.groupby(hospital_data.columns).count().where("count > 1").show() #check the duplicates

In [0]:
hospital_data = hospital_data.dropDuplicates()

In [0]:
hospital_data = hospital_data.replace('NaN', None)

In [0]:
#hospital_data.show(5,False)

In [0]:
hospital_data = hospital_data.fillna({"state":"UT"})

#hospital_data = hospital_data.fillna("UT", subset=["state"])

#hospital_data = hospital_data.na.fill(value="UT",subset=["state"])


In [0]:
#hospital_data.show(10,False)

In [0]:
output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
hospital_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/hospitaldatastagging.csv" % output_container_path)